In [2]:
from google.colab import drive
from tensorflow.keras.layers import Convolution3D,MaxPooling3D,Dense,Flatten
import tensorflow as tf
from tensorflow.keras.models import Model,Sequential
import numpy as np
import pandas as pd
from sklearn.externals import joblib
import os
import time
from google.colab.patches import cv2_imshow
import random
from IPython.core.debugger import set_trace
from google.colab import output


/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [3]:
drive.mount("gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at gdrive


In [4]:
porn_path = "gdrive/My Drive/VideoExtractedFrames/Backup/PFrames/"
porn_list = os.listdir(porn_path)
print(len(porn_list))
#print(porn_list[-100])
#print(porn_list[-50])

np_path = "gdrive/My Drive/VideoExtractedFrames/Backup/NPFrames/"
np_list = os.listdir(np_path)
print(len(np_list))
#print(np_list[-50])
#print(np_list[-25])

npd_path = "gdrive/My Drive/VideoExtractedFrames/Backup/NPDFrames/"
npd_list = os.listdir(npd_path)
print(len(npd_list))
#print(npd_list[-50])
#print(npd_list[-25])

0
0
0


In [5]:
train_path = "gdrive/My Drive/VideoExtractedFrames/ModelData/train/"
test_path = "gdrive/My Drive/VideoExtractedFrames/ModelData/test/"
val_path = "gdrive/My Drive/VideoExtractedFrames/ModelData/val/"
train_list = os.listdir(train_path)
test_list = os.listdir(test_path)
val_list = os.listdir(val_path)
print(train_list)
print(test_list)
print(val_list)


['porn', 'nonporn']
['porn', 'nonporn']
['porn', 'nonporn']


In [0]:
label_dict = {
    "porn":0,
    "nonporn":1
}

In [0]:
def AddressResolve(path,label_dict):
    porn_p = os.path.join(path,"porn")
    nonporn_p = os.path.join(path,"nonporn")
    p_folders = os.listdir(porn_p)
    np_folders = os.listdir(nonporn_p)
    folder_address_dict = {}
    for _item in p_folders:
        print("Accessing Folder : ",_item)
        folder_address_dict[_item] = {}
        folder_address_dict[_item]["label"] = label_dict["porn"]
        folder_address_dict[_item]["address"] = os.path.join(porn_p,_item)
        #folder_address_dict[_item]["frames"] = os.listdir(temp_add)
    for _item in np_folders:
        print("Accessing Folder : ",_item)
        folder_address_dict[_item] = {}
        folder_address_dict[_item]["label"] = label_dict["nonporn"]
        folder_address_dict[_item]["address"] = os.path.join(nonporn_p,_item)
        #folder_address_dict[_item]["frames"] = os.listdir(temp_add)
    return folder_address_dict    

def AddingFrames(folder_address_dict):
    erratic = []
    for key in list(folder_address_dict.keys()):
        print("Accessing Inner Frames : ",key)
        try:
            folder_address_dict[key]["frames"] = os.listdir(folder_address_dict[key]["address"])
            np.save("gdrive/My Drive/SupportingFiles/info_dict.npy",folder_address_dict)
        except:
            #output.clear()
            print("Clearing Output Mode : ",key)
            erratic.append(key)
            print(erratic)
            #folder_address_dict[key]["frames"] = os.listdir(folder_address_dict[key]["address"]) 
            #print("Accessing Done")     
    return folder_address_dict,erratic              

In [0]:
train_dict = AddressResolve(train_path,label_dict)
try:
    train_dict = np.load("gdrive/My Drive/SupportingFiles/info_dict.npy")
except:
    print("Not Loaded")    
#output.clear()
train_dict,err = AddingFrames(train_dict)
print(err)

Accessing Folder :  frames_vPorn1
Accessing Folder :  frames_vPorn2
Accessing Folder :  frames_vPorn3
Accessing Folder :  frames_vPorn4
Accessing Folder :  frames_vPorn5
Accessing Folder :  frames_vPorn6
Accessing Folder :  frames_vPorn8
Accessing Folder :  frames_vPorn10
Accessing Folder :  frames_vPorn7
Accessing Folder :  frames_vPorn11
Accessing Folder :  frames_vPorn13
Accessing Folder :  frames_vPorn14
Accessing Folder :  frames_vPorn15
Accessing Folder :  frames_vPorn16
Accessing Folder :  frames_vPorn17
Accessing Folder :  frames_vPorn18
Accessing Folder :  frames_vPorn19
Accessing Folder :  frames_vPorn20
Accessing Folder :  frames_vPorn21
Accessing Folder :  frames_vPorn22
Accessing Folder :  frames_vPorn23
Accessing Folder :  frames_vPorn24
Accessing Folder :  frames_vPorn25
Accessing Folder :  frames_vPorn26
Accessing Folder :  frames_vPorn29
Accessing Folder :  frames_vPorn30
Accessing Folder :  frames_vPorn31
Accessing Folder :  frames_vPorn32
Accessing Folder :  frames_v

In [0]:
data_dict = np.load("gdrive/My Drive/SupportingFiles/info_dict.npy",allow_pickle = True)

In [0]:
data_dictionary  = data_dict.item()

In [35]:
def SortingFrames(dictionary):
    for key in dictionary.keys():
        items = list(dictionary[key]["frames"])
        print(items)
        items = sorted(items,key = lambda x: int(x.split("_")[1]))
        dictionary[key]["frames"] = items
    return dictionary

'\ndef SortingFrames(dictionary):\n    for key in dictionary.keys():\n        items = list(dictionary[key]["frames"])\n        print(items)\n        items = sorted(items,key = lambda x: int(x.split("_")[1]))\n        dictionary[key]["frames"] = items\n    return dictionary\n'

In [31]:
final_dict = SortingFrames(data_dictionary)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

